# Import Library

In [15]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import MobileNet_V3_Small_Weights
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, r2_score
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [16]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device} as device.")

Using cpu as device.


# Function Library

In [17]:
def train_model(model, train_loader, val_loader, y_scaler, epochs=5):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for imgs, _, idxs in tqdm(train_loader):
            imgs = imgs.to(device)
            labels = torch.tensor(bmis_scaled[idxs], dtype=torch.float32).unsqueeze(1).to(device)

            preds = model(imgs)
            loss = criterion(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f"\nEpoch {epoch+1}: Train Loss = {train_loss/len(train_loader):.4f}")

        # Validation
        model.eval()
        val_preds, val_labels = [], []

        with torch.no_grad():
            for imgs, _, idxs in val_loader:
                imgs = imgs.to(device)
                labels = torch.tensor(bmis_scaled[idxs], dtype=torch.float32).unsqueeze(1).to(device)

                preds = model(imgs)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        # Convert back to original BMI units
        val_preds_real = y_scaler.inverse_transform(np.array(val_preds).reshape(-1, 1)).ravel()
        val_labels_real = y_scaler.inverse_transform(np.array(val_labels).reshape(-1, 1)).ravel()

        mae = mean_absolute_error(val_labels_real, val_preds_real)
        r2 = r2_score(val_labels_real, val_preds_real)
        r, _ = pearsonr(val_labels_real, val_preds_real)
        print(f"Val MAE: {mae:.2f} | Val R²: {r2:.3f} | Pearson r: {r:.3f}")


# Class Library

In [18]:
class BMIDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform
        
        # Remove any rows without image files
        self.df['full_path'] = self.df['name'].apply(lambda x: os.path.join(image_dir, x))
        self.df = self.df[self.df['full_path'].apply(os.path.exists)].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_path']).convert('RGB')

        if self.transform:
            img = self.transform(img)

        label = row['bmi']
        return img, label, idx 

# Dataset Transformation

In [19]:
# Basic transform
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Normalized to [-1, 1]
])

# Dataset
dataset = BMIDataset(
    csv_path='landmark_features.csv',
    image_dir='../data/BMI/Images',
    transform=img_transform
)

# Split dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [20]:
# Load BMI column and scale
df = pd.read_csv('landmark_features.csv')
bmis = df['bmi'].values
y_scaler = StandardScaler()
bmis_scaled = y_scaler.fit_transform(bmis.reshape(-1, 1)).ravel()

In [21]:
# Load pretrained model with proper weights argument
weights = MobileNet_V3_Small_Weights.DEFAULT
model = mobilenet_v3_small(weights=weights)

# Fully replace classifier for regression
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[0].in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 1)
)

In [22]:
# Loss: Mean Squared Error for regression
criterion = nn.MSELoss()

In [23]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [24]:
train_model(model, train_loader, val_loader, y_scaler, epochs= 5)

100%|██████████| 99/99 [06:56<00:00,  4.21s/it]



Epoch 1: Train Loss = 0.9689
Val MAE: 5.72 | Val R²: 0.127 | Pearson r: 0.448


100%|██████████| 99/99 [07:07<00:00,  4.31s/it]



Epoch 2: Train Loss = 0.7339
Val MAE: 5.07 | Val R²: 0.296 | Pearson r: 0.553


100%|██████████| 99/99 [09:01<00:00,  5.47s/it]



Epoch 3: Train Loss = 0.5622
Val MAE: 4.90 | Val R²: 0.338 | Pearson r: 0.584


100%|██████████| 99/99 [07:59<00:00,  4.84s/it]



Epoch 4: Train Loss = 0.4479
Val MAE: 5.69 | Val R²: 0.211 | Pearson r: 0.580


100%|██████████| 99/99 [10:49<00:00,  6.56s/it]



Epoch 5: Train Loss = 0.3314
Val MAE: 4.91 | Val R²: 0.324 | Pearson r: 0.580


In [14]:
torch.save(model.state_dict(), "cnn_model_final.pt")